In [30]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np

In [6]:
pilares = gpd.read_file('datos/insumos/pilares.geojson')
coberturas = gpd.read_file('datos/insumos/isodistancia_750m_pilares.geojson')
manzanas = gpd.read_file('C:/SCINCE 2020/09_CDMX/cartografia/manzana.shp')
print(pilares.crs)
print(coberturas.crs)
print(manzanas.crs)

EPSG:4326
EPSG:32614
EPSG:6372


In [3]:
pilares.head(1)

,nombre,alcaldia,direccion,latitud,longitud,horario,geometry
0,100 Metros,Gustavo A Madero,"Av Cien Metros y Anillo Periferico S/N , Santi...",19.526523,-99.158897,De 09:00 a 19:00,POINT (-99.1589 19.52652)


In [4]:
coberturas.head(1)

,node,nombre,geometry
0,30594067,Benito Juárez Y El Liberalismo Mexicano,"POLYGON ((483148.725 2146147.127, 483147.744 2..."


In [7]:
manzanas.head(1)

,CVEGEO,POB1,POB2,POB2_R,POB4,POB4_R,POB5,POB5_R,POB6,POB6_R,...,POB121,POB121_R,POB122,POB122_R,POB123,POB123_R,POB124,POB124_R,OID,geometry
0,0901000010898031,93.0,4.0,4.3,-6.0,-6.0,5.0,5.4,9.0,9.7,...,6.0,6.5,-6.0,-6.0,0.0,0.0,-6.0,-6.0,1,"POLYGON ((2790204.135 820779.247, 2790125.593 ..."


# Identificar columnas de ínteres de la capa de manznas

In [9]:
# cargar archivo de descriptores
desc = pd.read_excel('C:/SCINCE 2020/Descriptores_2020.xlsx')
desc.head(1)

,CAMPO,DESCRIP
0,POB1,Población total


In [13]:
cdi = ['POB1', 'POB2', 'POB4', 'POB5', 'POB7', 'POB9','POB13', 'POB14', 'POB15', 'POB16',  'POB24', 'POB42', 'POB84', 'MIG7', 'DISC1', 'ECO25', 'EDU49_R', 'EDU50_R','EDU51_R'
]
desc_cdi = desc[desc['CAMPO'].isin(cdi)].copy()
desc_cdi

,CAMPO,DESCRIP
0,POB1,Población total
1,POB2,Población de 0 a 2 años
3,POB4,Población de 3 a 5 años
4,POB5,Población de 6 a 11 años
6,POB7,Población de 12 a 14 años
8,POB9,Población de 15 a 17 años
12,POB13,Población de 18 a 24 años
13,POB14,Población de 30 a 49 años
14,POB15,Población de 50 a 59 años
15,POB16,Población de 60 a 64 años


In [23]:
# Create a mapping dictionary
column_mapping = {
    'POB1': 'Pobtotal',
    'POB2': 'edad_0-2',
    'POB4': 'edad_3-5',
    'POB5': 'edad_6-11',
    'POB7': 'edad_12-14',
    'POB9': 'edad_15-17',
    'POB13': 'edad_18-24',
    'POB14': 'edad_30-49',
    'POB15': 'edad_50-59',
    'POB16': 'edad_60-64',
    'POB24': 'edad_65+',
    'POB42': 'Mujeres',
    'POB84': 'Hombres',
    'MIG7': 'NacidoOtroPaís',
    'DISC1': 'PobConDiscapacidad',
    'ECO25': 'PobDesocupada',
    'EDU49_R': 'EscoProm',
    'EDU50_R': 'EscoPromMujeres',
    'EDU51_R': 'EscoPromHombres'
}

# Apply the mapping to create new column
desc_cdi['DESCRIP_SHORT'] = desc_cdi['CAMPO'].map(column_mapping)
desc_cdi.head()

,CAMPO,DESCRIP,DESCRIP_SHORT
0,POB1,Población total,Pobtotal
1,POB2,Población de 0 a 2 años,edad_0-2
3,POB4,Población de 3 a 5 años,edad_3-5
4,POB5,Población de 6 a 11 años,edad_6-11
6,POB7,Población de 12 a 14 años,edad_12-14


In [16]:
# Unimos información de tablas a las manzanas
mig = pd.read_csv("C:/SCINCE 2020/09_CDMX/tablas_csv/cpv2020_manzana_migracion.csv")
disc = pd.read_csv("C:/SCINCE 2020/09_CDMX/tablas_csv/cpv2020_manzana_discapacidad.csv")
eco = pd.read_csv("C:/SCINCE 2020/09_CDMX/tablas_csv/cpv2020_manzana_caracteristicas_economicas.csv")
edu = pd.read_csv("C:/SCINCE 2020/09_CDMX/tablas_csv/cpv2020_manzana_educacion.csv")

print(mig.head(1))
print(disc.head(1))
print(eco.head(1))
print(edu.head(1))

             CVEGEO  MIG1  MIG1_R  MIG2  MIG2_R  MIG3  MIG3_R  MIG4  MIG4_R  \
0  0900200010010001   138    86.8    74    86.0    64    87.7    21    13.2   

   MIG5  ...  MIG11  MIG11_R  MIG12  MIG12_R  MIG13  MIG13_R  MIG14  MIG14_R  \
0    12  ...      0      0.0      0      0.0      0      0.0      0      0.0   

   MIG15  MIG15_R  
0      0      0.0  

[1 rows x 31 columns]
             CVEGEO  DISC1  DISC1_R  DISC2  DISC2_R  DISC3  DISC3_R  DISC4  \
0  0900200010010001     11      6.9      3      3.5      8     11.0      0   

   DISC4_R  DISC5  ...  DISC16  DISC16_R  DISC17  DISC17_R  DISC18  DISC18_R  \
0      0.0      0  ...       9      10.5      11      15.1       0       0.0   

   DISC19  DISC19_R  DISC20  DISC20_R  
0       7       6.6      13      30.2  

[1 rows x 41 columns]
             CVEGEO  ECO1  ECO1_R  ECO2  ECO2_R  ECO3  ECO3_R  ECO4  ECO4_R  \
0  0900200010010001    81    53.6    32    41.0    49    67.1    77    95.1   

   ECO5  ...  ECO31  ECO31_R  ECO34  

In [34]:
# Get the list of columns you want from desc_cdi
columns_to_keep = desc_cdi['CAMPO'].tolist()
# Filter manzanas to keep only CVEGEO + columns from desc_cdi that exist in it
manzanas_cols = ['CVEGEO', 'geometry'] + [col for col in columns_to_keep if col in manzanas.columns]
result = manzanas[manzanas_cols].copy()
# Dictionary of tables
tables_dict = {
    'mig': mig,
    'disc': disc,
    'eco': eco,
    'edu': edu
}


# Merge each table
for name, table in tables_dict.items():
    # Find which columns from desc_cdi exist in this table
    cols = ['CVEGEO'] + [col for col in columns_to_keep if col in table.columns]
    result = result.merge(table[cols], on='CVEGEO', how='left')
    print(f"Merged {name}: {len(cols)-1} columns")
    
result.head(1)


Merged mig: 1 columns
Merged disc: 1 columns
Merged eco: 1 columns
Merged edu: 3 columns


,CVEGEO,geometry,POB1,POB2,POB4,POB5,POB7,POB9,POB13,POB14,...,POB16,POB24,POB42,POB84,MIG7,DISC1,ECO25,EDU49_R,EDU50_R,EDU51_R
0,0901000010898031,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",93.0,4.0,-6.0,5.0,5.0,3.0,8.0,27.0,...,5.0,19.0,56.0,37.0,0,5,3,10.6,10.7,10.4


In [35]:
# Creamos un diccionario con los nuevos nombres de las columnas
rename = dict(zip(desc_cdi['CAMPO'], desc_cdi['DESCRIP_SHORT']))

result = result.rename(columns=rename)
result.head(1)


,CVEGEO,geometry,Pobtotal,edad_0-2,edad_3-5,edad_6-11,edad_12-14,edad_15-17,edad_18-24,edad_30-49,...,edad_60-64,edad_65+,Mujeres,Hombres,NacidoOtroPaís,PobConDiscapacidad,PobDesocupada,EscoProm,EscoPromMujeres,EscoPromHombres
0,0901000010898031,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",93.0,4.0,-6.0,5.0,5.0,3.0,8.0,27.0,...,5.0,19.0,56.0,37.0,0,5,3,10.6,10.7,10.4


In [36]:
result.describe()

,Pobtotal,edad_0-2,edad_3-5,edad_6-11,edad_12-14,edad_15-17,edad_18-24,edad_30-49,edad_50-59,edad_60-64,edad_65+,Mujeres,Hombres,NacidoOtroPaís,PobConDiscapacidad,PobDesocupada,EscoProm,EscoPromMujeres,EscoPromHombres
count,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000,66386.000000
mean,137.656765,1.210165,2.296282,8.660757,3.106197,3.361944,13.441961,40.957476,16.172612,5.124198,14.116169,71.170819,65.064637,-0.368255,5.348899,-1.345977,10.140653,9.927940,10.382897
std,189.013505,8.292716,9.327647,16.463271,9.850089,10.015586,21.481541,67.587385,24.281187,10.977196,20.323021,95.422724,99.558353,10.320225,12.265962,5.094432,5.371897,5.310945,5.499735
min,0.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000
25%,43.000000,-6.000000,-6.000000,0.000000,-6.000000,-6.000000,4.000000,12.000000,5.000000,-6.000000,4.000000,22.000000,20.000000,-6.000000,-6.000000,-6.000000,9.800000,9.600000,9.900000
50%,92.000000,0.000000,3.000000,6.000000,3.000000,3.000000,9.000000,27.000000,11.000000,5.000000,10.000000,48.000000,44.000000,0.000000,4.000000,0.000000,11.100000,10.900000,11.300000
75%,169.000000,5.000000,6.000000,13.000000,7.000000,7.000000,18.000000,50.000000,22.000000,9.000000,20.000000,88.000000,81.000000,0.000000,9.000000,0.000000,12.900000,12.600000,13.200000
max,10484.000000,298.000000,381.000000,712.000000,381.000000,462.000000,1043.000000,6220.000000,1449.000000,582.000000,1156.000000,5532.000000,8515.000000,1122.000000,476.000000,172.000000,18.800000,20.000000,22.000000


In [37]:
#  remplazamos valores que no aplican por 0 y confidenciales por NaN
result = result.replace(-6, 0)
result = result.replace(-8, np.nan)

In [41]:
print(result.crs)
print(coberturas.crs)

EPSG:6372
EPSG:32614


In [43]:
result.columns

Index(['CVEGEO', 'geometry', 'Pobtotal', 'edad_0-2', 'edad_3-5', 'edad_6-11',
       'edad_12-14', 'edad_15-17', 'edad_18-24', 'edad_30-49', 'edad_50-59',
       'edad_60-64', 'edad_65+', 'Mujeres', 'Hombres', 'NacidoOtroPaís',
       'PobConDiscapacidad', 'PobDesocupada', 'EscoProm', 'EscoPromMujeres',
       'EscoPromHombres'],
      dtype='object')

# Union espacial

In [44]:
agregacion = {}
# Columnas de poblacion total y por edad
pop_columns = ['Pobtotal', 'edad_0-2', 'edad_3-5', 'edad_6-11',
       'edad_12-14', 'edad_15-17', 'edad_18-24', 'edad_30-49', 'edad_50-59',
       'edad_60-64', 'edad_65+', 'Mujeres', 'Hombres', 'NacidoOtroPaís',
       'PobConDiscapacidad', 'PobDesocupada']
for col in pop_columns:
    agregacion[col] = 'sum'
# Columnas de escolaridad promedio
esco_columns = ['EscoProm', 'EscoPromMujeres', 'EscoPromHombres']

for col in esco_columns:
    agregacion[col] = 'mean'
    
sjoin = coberturas.sjoin(result.to_crs(epsg="32614"), how='left', predicate='intersects')
print(f"Coberturas original: {len(coberturas)}")
print(f"After spatial join: {len(sjoin)}")

sjoin_agg = sjoin.groupby(sjoin.index).agg(agregacion)

result_final = coberturas.join(sjoin_agg)

print(f"After aggregation: {len(result_final)}")
print(f"\nColumns in final result: {result_final.columns.tolist()}")

result_final.head()

Coberturas original: 303
After spatial join: 32741
After aggregation: 303

Columns in final result: ['node', 'nombre', 'geometry', 'Pobtotal', 'edad_0-2', 'edad_3-5', 'edad_6-11', 'edad_12-14', 'edad_15-17', 'edad_18-24', 'edad_30-49', 'edad_50-59', 'edad_60-64', 'edad_65+', 'Mujeres', 'Hombres', 'NacidoOtroPaís', 'PobConDiscapacidad', 'PobDesocupada', 'EscoProm', 'EscoPromMujeres', 'EscoPromHombres']


,node,nombre,geometry,Pobtotal,edad_0-2,edad_3-5,edad_6-11,edad_12-14,edad_15-17,edad_18-24,...,edad_60-64,edad_65+,Mujeres,Hombres,NacidoOtroPaís,PobConDiscapacidad,PobDesocupada,EscoProm,EscoPromMujeres,EscoPromHombres
0,30594067,Benito Juárez Y El Liberalismo Mexicano,"POLYGON ((483148.725 2146147.127, 483147.744 2...",10946.0,138.0,210.0,395.0,148.0,208.0,803.0,...,579.0,1375.0,5410.0,5524.0,876.0,463.0,141.0,13.505333,13.213333,13.829333
1,30595404,San Lorenzo Huipulco,"POLYGON ((484940.227 2133742.777, 484939.245 2...",7173.0,151.0,192.0,534.0,285.0,297.0,781.0,...,359.0,886.0,3760.0,3406.0,43.0,339.0,70.0,12.231707,12.007317,12.490244
2,268533822,100 Metros,"POLYGON ((483344.762 2158917.58, 483343.781 21...",7705.0,150.0,200.0,532.0,241.0,269.0,795.0,...,432.0,1116.0,4022.0,3674.0,18.0,472.0,79.0,12.203175,11.906349,12.526984
3,268822705,Ratón Macías,"POLYGON ((484378.627 2145124.704, 484377.649 2...",27509.0,692.0,781.0,1817.0,953.0,883.0,2605.0,...,1360.0,2810.0,14239.0,13158.0,484.0,1112.0,283.0,11.809565,11.566087,12.020870
4,268838941,Frida Kahlo,"POLYGON ((485059.508 2145822.803, 485058.529 2...",24694.0,644.0,762.0,1667.0,858.0,898.0,2372.0,...,1407.0,3170.0,12954.0,11731.0,158.0,1567.0,296.0,11.163793,10.874138,11.515517


In [47]:
# Calcular porcentajes
# List of columns to convert to percentages
columns_for_pct = ['edad_0-2', 'edad_3-5', 'edad_6-11', 'edad_12-14', 
                   'edad_15-17', 'edad_18-24', 'edad_30-49', 'edad_50-59',
                   'edad_60-64', 'edad_65+', 'Mujeres', 'Hombres', 
                   'NacidoOtroPaís', 'PobConDiscapacidad', 'PobDesocupada']

# Calculate percentages
for col in columns_for_pct:
    result_final[f'pct_{col}'] = ((result_final[col] / result_final['Pobtotal']) * 100).round(1)

# Replace inf and NaN values (in case Pobtotal is 0 somewhere)
result_final = result_final.replace([np.inf, -np.inf], np.nan)

print("Percentage columns created!")
result_final.head(3)

Percentage columns created!


,node,nombre,geometry,Pobtotal,edad_0-2,edad_3-5,edad_6-11,edad_12-14,edad_15-17,edad_18-24,...,pct_edad_18-24,pct_edad_30-49,pct_edad_50-59,pct_edad_60-64,pct_edad_65+,pct_Mujeres,pct_Hombres,pct_NacidoOtroPaís,pct_PobConDiscapacidad,pct_PobDesocupada
0,30594067,Benito Juárez Y El Liberalismo Mexicano,"POLYGON ((483148.725 2146147.127, 483147.744 2...",10946.0,138.0,210.0,395.0,148.0,208.0,803.0,...,7.3,40.2,12.3,5.3,12.6,49.4,50.5,8.0,4.2,1.3
1,30595404,San Lorenzo Huipulco,"POLYGON ((484940.227 2133742.777, 484939.245 2...",7173.0,151.0,192.0,534.0,285.0,297.0,781.0,...,10.9,30.3,12.4,5.0,12.4,52.4,47.5,0.6,4.7,1.0
2,268533822,100 Metros,"POLYGON ((483344.762 2158917.58, 483343.781 21...",7705.0,150.0,200.0,532.0,241.0,269.0,795.0,...,10.3,29.2,12.4,5.6,14.5,52.2,47.7,0.2,6.1,1.0


In [50]:
pilares.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [51]:
result_final.to_crs(epsg="4326").to_file('../src/assets/data/coberturas_pilares_datos.geojson', driver='GeoJSON')
pilares.to_crs(epsg="4326").to_file('../src/assets/data/pilares.geojson', driver='GeoJSON')